In [ ]:
'''
Group14

Team Members:
Gayathri Punukollu
Sushma Kumbham
Usha Sree Komati
Vijita Sagari Bobba

All members have contributed equally to the assignment'''

In [1108]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from random import seed
from random import randrange
from random import random
import math

In [1109]:
df = pd.read_csv('/Users/sushmakumbham/Downloads/data_banknote_authentication.csv', header = None)

In [1110]:
df.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [1111]:
df_list = df.values.tolist()

#Since list indices should not have float values, we are converting it into int
for i in range(len(df_list)):
    df_list[i][4] = int(df_list[i][4])

In [1112]:
#Splitting the train,test data
train_data, test_data = train_test_split(df_list, test_size=0.33, random_state=42)

In [1113]:
#Function to calculate the accuracy
def get_accuracy(true, pred):
    count = 0
    l=len(true)
    x=float(l)
    for i in range(l):
        if true[i] == pred[i]:
            count = count+1
    val=count/x
    return val * 100.0

In [1114]:
#Each splitted data function
def each_split(fold,each_fold_len,df_list):
    while len(fold) < each_fold_len:
        index = randrange(len(df_list))
        fold.append(df_list.pop(index))
    return fold

In [1115]:
# Dataset splitting using k folds
def kfold_split(df_list,lenx, k_folds):
    data_splitted = list()
    each_fold_len = int(lenx / k_folds)
    for i in range(k_folds):
        fold = list()
        fold = each_split(fold,each_fold_len,df_list)
        data_splitted.append(fold)
    return data_splitted

In [1116]:
#Obtaining all the split data needed to calculate the accuracy
def calc_acc(algo,i,train_set_data,test_set_data, *args):
    for each in i:
        each_list = list(each)
        test_set_data.append(each_list)
        each_list[-1] = None
    pred = algo(train_set_data, test_set_data, *args)
    true = [each[-1] for each in i]
    accuracy = get_accuracy(true, pred)
    return accuracy

In [1117]:
#Cross validation split for each method we perform
def algorithm_cvs(df_list,lenx, algo, k_folds, *args):
    Kfold = kfold_split(df_list,lenx,k_folds)
    acc = list()
    for i in Kfold:
        test_set_data = list()
        train_set_data = list(Kfold)
        train_set_data.remove(i)
        train_set_data = sum(train_set_data, [])
        accuracy=calc_acc(algo,i,train_set_data,test_set_data, *args)
        acc.append(accuracy)
    return acc

In [1118]:
# Initializing a network
def get_initial_network(no_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = []
    output_layer = []
    for i in range(n_hidden):
        weights = {'weights': []}
        for j in range(no_inputs + 1):
            weights['weights'].append(random())
        hidden_layer.append(weights) 
    network.append(hidden_layer)
    for i in range(n_outputs):
        weights = {'weights': []}
        for j in range(n_hidden + 1):
            weights['weights'].append(random())
        output_layer.append(weights)
    network.append(output_layer)
    return network

In [1119]:
#For a given input, we are calculating the neuron activation function
def neu_activation(weights, ip):
    activation = weights[-1]
    aw=len(weights)-1
    for i in range(aw):
        aw_ip=weights[i] * ip[i]
        activation = activation+aw_ip
    return activation

In [1120]:
#Weights updating function
def wu(i,net,inputx,learning_rate):
    for neu in net[i]:
        y=len(inputx)
        for j in range(y):
            neu['weights'][j] += learning_rate * neu['delta'] * inputx[j]
        neu['weights'][-1] += learning_rate * neu['delta']

In [1121]:
# Weight updation by getting the error value
def weights_updation(net, r, learning_rate):
    x=len(net)
    for i in range(x):
        inputx = r[:-1]
        if i != 0:
            inputx = [neu['output'] for neu in net[i - 1]]
        wu(i,net,inputx,learning_rate)

In [1122]:
# Computing the Predictions for sigmoid while performing the forward propagation for the data.
def sigmoid_predict(net, r):
    i = r
    for layer in net:
        new_ip = []
        for neu in layer:
            activation = neu_activation(neu['weights'], i)
            neu['output'] = 1.0 / (1.0 + np.exp(-activation))
            new_ip.append(neu['output'])
        i = new_ip
    a = i
    return a.index(max(a))

In [1123]:
#Network train initialization with a given number of epochs
def Train_net_sigmoid_func(net, train, learning_rate, n_epochs, n_op):
    for epochs in range(n_epochs):
        for i in train:
            op = sigmoid_predict(net, i)
            exp = [0 for i in range(n_op)]
            exp[i[-1]] = 1
            back_propagate_sigmod_func(net, exp)
            weights_updation(net, i, learning_rate)

In [1124]:
#Updating the delta weights function
def update_delta_weights_sigmoid(layer,errors):
    for j in range(len(layer)):
            neu = layer[j]
            neu['delta'] = errors[j] * (neu['output'] * (1.0 - neu['output']))

In [1125]:
#Error calculating function
def get_errors(i,x,layer,errors,net,exp):
    if i != x-1:
        for j in range(len(layer)):
            error = 0.0
            for neu in net[i + 1]:
                error += (neu['weights'][j] * neu['delta'])
            errors.append(error)
    else:
        for j in range(len(layer)):
            neu = layer[j]
            errors.append(exp[j] - neu['output'])
    return errors

In [1126]:
# Function to calculate the backpropagation error for sigmoid derivation
def back_propagate_sigmod_func(net, exp):
    x=len(net)
    for i in reversed(range(x)):
        layer = net[i]
        layer_length=len(layer)
        errors = list()
        errors=get_errors(i,x,layer,errors,net,exp)
        update_delta_weights_sigmoid(layer,errors)

In [1127]:
#Function for backpropagation - sigmoid
def Back_propagation_sigmoid_algo(train, test, l_rate, n_epochs, n_hid):
    trainLen=len(train[0])
    no_inputs = trainLen - 1
    no_outputs = len(set([value[-1] for value in train]))
    network = get_initial_network(no_inputs, n_hid, no_outputs)
    Train_net_sigmoid_func(network, train, l_rate, n_epochs, no_outputs)
    pred_list = list()
    for value in test:
        pred = sigmoid_predict(network, value)
        pred_list.append(pred)
    return(pred_list)

In [1128]:
k_folds = 5
print('\n Number of k-folds =', k_folds)
learning_rate = 0.001
epochs = 100
i=6
while i > 0 :
    train_dlist = list(train_data)
    lenx = len(train_data)
    sigmoid_val = algorithm_cvs(train_dlist,lenx, Back_propagation_sigmoid_algo, k_folds, learning_rate, epochs, i)
    print('\n Hidden Layer Nodes =', i)
    print('Accuracy of k-folds = ', sigmoid_val)
    mean_accuracy = sum(sigmoid_val)/float(len(sigmoid_val))
    print('Mean Accuracy = ', mean_accuracy)
    i = i-1



 Number of k-folds = 5

 Hidden Layer Nodes = 6
Accuracy of k-folds =  [96.17486338797814, 91.2568306010929, 92.89617486338798, 94.53551912568307, 95.08196721311475]
Mean Accuracy =  93.98907103825137

 Hidden Layer Nodes = 5
Accuracy of k-folds =  [94.53551912568307, 94.53551912568307, 91.2568306010929, 95.62841530054644, 95.62841530054644]
Mean Accuracy =  94.31693989071036

 Hidden Layer Nodes = 4
Accuracy of k-folds =  [94.53551912568307, 98.36065573770492, 93.44262295081968, 91.80327868852459, 90.1639344262295]
Mean Accuracy =  93.66120218579235

 Hidden Layer Nodes = 3
Accuracy of k-folds =  [95.62841530054644, 92.89617486338798, 95.62841530054644, 92.89617486338798, 93.44262295081968]
Mean Accuracy =  94.09836065573771

 Hidden Layer Nodes = 2
Accuracy of k-folds =  [95.08196721311475, 93.98907103825137, 97.81420765027322, 95.62841530054644, 92.89617486338798]
Mean Accuracy =  95.08196721311474

 Hidden Layer Nodes = 1
Accuracy of k-folds =  [98.36065573770492, 93.4426229508196

In [1129]:
#The best accuracy is found with hidden layer node 2. Lets test with test dataset.

In [1144]:
learning_rate = 0.001
epochs = 100

test_dlist = list(test_data)
lenx=len(test_data)
sig_val = algorithm_cvs(test_dlist,lenx, Back_propagation_sigmoid_algo, k_folds, learning_rate, epochs, 2)
print('\n Hidden Layer Nodes =', 2)
print('Accuracy of k-folds = ', sig_val)
mean_acc = sum(sig_val)/float(len(sig_val))
print('Mean Accuracy = ', mean_acc)


 Hidden Layer Nodes = 2
Accuracy of k-folds =  [98.88888888888889, 90.0, 71.11111111111111, 84.44444444444444, 91.11111111111111]
Mean Accuracy =  87.11111111111111


In [1131]:
#Hyperbolic tangent

In [1132]:
# performing the forward propagation for the given data.
def tan_predict(net, r):
    input_ = r
    for layer in net:
        new_ip = []
        for neu in layer:
            activation = neu_activation(neu['weights'], input_)
            neu['output']= (np.exp(activation) - np.exp(-activation))/(np.exp(activation) + np.exp(-activation))
            new_ip.append(neu['output'])
        input_ = new_ip
    a=input_
    return a.index(max(a))

In [1133]:
#Updating the delta weights function for tan
def update_delta_weights_tan(layer,errors):
    for j in range(len(layer)):
            neu = layer[j]
            sq=math.pow(neu['output'],2)
            neu['delta'] = errors[j] * (1.0 -sq)

In [1134]:
def backward_propagation_tan(net, exp):
    t= reversed(range(len(net)))
    x=len(net)
    k=x-1
    for i in t:
        layer = net[i]
        errors = list()
        len_layer=len(layer)
        errors=get_errors(i,x,layer,errors,net,exp)
        update_delta_weights_tan(layer,errors)

In [1135]:
# Initializing a network Train with a fixed number of epochs
def Train_net_tan(net, train, l_rate, n_epo, n_op):
    for epo in range(n_epo):
        for i in train:
            op = tan_predict(net, i)
            exp = [0 for i in range(n_op)]
            exp[i[-1]] = 1
            backward_propagation_tan(net, exp)
            weights_updation(net, i, l_rate)

In [1136]:
# Backpropagation Algorithm for tangent
def Back_propagation_algo_tan(train, test, l_rate, n_epo, n_hidden):
    no_inputs = len(train[0]) - 1
    no_outputs = len(set([value[-1] for value in train]))
    network = get_initial_network(no_inputs, n_hidden, no_outputs)
    Train_net_tan(network, train, l_rate, n_epo, no_outputs)
    pred_list = list()
    for value in test:
        pred = tan_predict(network, value)
        pred_list.append(pred)
    return(pred_list)

In [1137]:
k_folds = 5
print('\n Number of k-folds =', k_folds)
learning_rate = 0.001
epochs = 100

i = 6
while i>0 :
    train_dlist = list(train_data)
    lenx=len(train_data)
    tanh_value = algorithm_cvs(train_dlist, lenx,Back_propagation_algo_tan, k_folds, learning_rate, epochs, i)
    print('\n Hidden Layer Nodes =', i)
    print('Accuracy of k-folds = ', tanh_value)
    mean_accuracy = sum(tanh_value)/float(len(tanh_value))
    print('Mean Accuracy = ', mean_accuracy)
    i = i-1


 Number of k-folds = 5

 Hidden Layer Nodes = 6
Accuracy of k-folds =  [97.81420765027322, 98.36065573770492, 100.0, 99.4535519125683, 98.90710382513662]
Mean Accuracy =  98.9071038251366

 Hidden Layer Nodes = 5
Accuracy of k-folds =  [99.4535519125683, 98.36065573770492, 98.90710382513662, 100.0, 97.26775956284153]
Mean Accuracy =  98.79781420765028

 Hidden Layer Nodes = 4
Accuracy of k-folds =  [97.26775956284153, 99.4535519125683, 100.0, 98.36065573770492, 99.4535519125683]
Mean Accuracy =  98.9071038251366

 Hidden Layer Nodes = 3
Accuracy of k-folds =  [100.0, 98.90710382513662, 97.81420765027322, 98.90710382513662, 99.4535519125683]
Mean Accuracy =  99.01639344262296

 Hidden Layer Nodes = 2
Accuracy of k-folds =  [99.4535519125683, 98.90710382513662, 98.90710382513662, 97.81420765027322, 98.36065573770492]
Mean Accuracy =  98.68852459016394

 Hidden Layer Nodes = 1
Accuracy of k-folds =  [98.36065573770492, 99.4535519125683, 99.4535519125683, 97.26775956284153, 100.0]
Mean Ac

In [1138]:
#The best accuracy is found with hidden layer node 3. Lets test  with test dataset.

In [1139]:
learning_rate = 0.001
epochs = 100

test_dlist = list(test_data)
lenx=len(test_data)
tanh_val = algorithm_cvs(test_dlist, lenx,Back_propagation_algo_tan, k_folds, learning_rate, epochs,3  )
print('\n Hidden Layer Nodes =', 3)
print('Accuracy of k-folds = ', tanh_value)
mean_accuracy = sum(tanh_val)/float(len(tanh_val))
print('Mean Accuracy = ', mean_accuracy)


 Hidden Layer Nodes = 3
Accuracy of k-folds =  [98.36065573770492, 99.4535519125683, 99.4535519125683, 97.26775956284153, 100.0]
Mean Accuracy =  98.66666666666667
